In [1]:
pip install opencv-python numpy


Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import numpy as np

# Function to read the video file
def read_video(file_path):
    cap = cv2.VideoCapture(file_path)
    return cap


In [2]:
def define_quadrants(frame):
    height, width, _ = frame.shape
    quadrants = {
        1: [(0, 0), (width//2, height//2)],
        2: [(width//2, 0), (width, height//2)],
        3: [(0, height//2), (width//2, height)],
        4: [(width//2, height//2), (width, height)]
    }
    return quadrants


In [3]:
def detect_balls(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define color ranges for detection
    colors = {
        'red': [(0, 120, 70), (10, 255, 255)],
        'green': [(36, 100, 100), (86, 255, 255)],
        'blue': [(94, 80, 2), (126, 255, 255)]
    }
    
    balls = []
    for color, (lower, upper) in colors.items():
        lower_np = np.array(lower)
        upper_np = np.array(upper)
        mask = cv2.inRange(hsv_frame, lower_np, upper_np)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 300:
                x, y, w, h = cv2.boundingRect(contour)
                balls.append((color, (x + w // 2, y + h // 2)))
    return balls


In [4]:
def record_event(events, time, quadrant, ball_color, event_type):
    event = f"{time:.2f}, {quadrant}, {ball_color}, {event_type}"
    events.append(event)

def is_in_quadrant(position, quadrant_coords):
    x, y = position
    (x1, y1), (x2, y2) = quadrant_coords
    return x1 <= x <= x2 and y1 <= y <= y2


In [5]:
def process_video(video_path, output_video_path, output_text_path):
    cap = read_video(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = None
    events = []

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    start_time = 0
    prev_positions = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if out is None:
            height, width, _ = frame.shape
            out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))
        
        quadrants = define_quadrants(frame)
        balls = detect_balls(frame)
        
        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
        
        for ball_color, position in balls:
            if ball_color not in prev_positions:
                prev_positions[ball_color] = {}

            for quadrant_num, quadrant_coords in quadrants.items():
                in_quadrant_now = is_in_quadrant(position, quadrant_coords)
                in_quadrant_prev = prev_positions[ball_color].get(quadrant_num, False)

                if in_quadrant_now and not in_quadrant_prev:
                    record_event(events, current_time, quadrant_num, ball_color, 'Entry')
                    cv2.putText(frame, f"Entry {current_time:.2f}", (position[0], position[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                elif not in_quadrant_now and in_quadrant_prev:
                    record_event(events, current_time, quadrant_num, ball_color, 'Exit')
                    cv2.putText(frame, f"Exit {current_time:.2f}", (position[0], position[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                prev_positions[ball_color][quadrant_num] = in_quadrant_now

        out.write(frame)

    cap.release()
    out.release()
    
    with open(output_text_path, 'w') as f:
        for event in events:
            f.write(f"{event}\n")


In [6]:
video_path = "C:/Users/Administrator/Downloads/AI Assignment video.mp4"
output_video_path = 'processed_video.avi'
output_text_path = 'events.txt'
process_video(video_path, output_video_path, output_text_path)
